<a href="https://colab.research.google.com/github/eswahn/stylegan2/blob/master/Stylegan2_RGBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# INITIALIZE

%tensorflow_version 1.x

!git clone https://github.com/eswahn/stylegan2.git
%cd stylegan2

!nvcc test_nvcc.cu -o test_nvcc -run
!nvidia-smi -L

TensorFlow 1.x selected.
Cloning into 'stylegan2'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 242 (delta 7), reused 0 (delta 0), pack-reused 222
Receiving objects: 100% (242/242), 2.32 MiB | 15.25 MiB/s, done.
Resolving deltas: 100% (129/129), done.
/content/stylegan2
CPU says hello.
GPU says hello.
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-766e4276-b953-4745-970d-62eed7fa2db8)


In [2]:
# LOAD DATASET

import math

dataset_name = 'andes-rgba'
dataset_path = '/content/drive/My\ Drive/Data/%s.tar.gz' % dataset_name     # path to tgz archive
result_dir = '/content/drive/My\ Drive/Results/stylegan2/' + dataset_name   # path to results

resolution = 512
min_w, min_h = 4, 4                                                         # square
resolution_n = int(math.log2(resolution/min_w))                             # re resolution, see skyflynil repo


!cp $dataset_path ./
!tar -xf *.tar.gz
!rm *.tar.gz
!mkdir dataset

In [ ]:
# PREPARE DATASET

!python dataset_tool.py create_from_images ./dataset/$dataset_name $dataset_name

Loading images from "andes-rgba"
detected 5302 images ...
Creating dataset "./dataset/andes-rgba"
Adding the images to tfrecords ...


In [ ]:
# START / RESUME TRAINING

!python run_training.py --use-raw=false --config=config-e --metric=none --num-gpus=1 --total-kimg=20000 \
                        --min-h=$min-h --min-w=$min-w --res-log2=$resolution_n --mirror-augment=true --mirror-augment-v=true \
                        --result-dir=$result_dir --data-dir=./dataset --dataset=$dataset_name 

In [ ]:
# TEST MODEL

import glob, os

network_dir = '/content/drive/My Drive/Results/stylegan2/' + dataset_name

allpickles = glob.glob(os.path.join(result_dir, '*', 'network-*.pkl'))
allpickles.sort(key=os.path.getctime)
pkl_path = '\"%s\"' % allpickles[-1]

!python run_generator.py generate-images --network=$pkl_path --seeds=0-100 --truncation-psi=1.0
